# Delitos Sexuales Transformation

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
PATH_FILE = '/content/drive/MyDrive/Police-Data/Bronce'
OUTPUT_PATH_FILE = '/content/drive/MyDrive/Police-Data/Silver'

## Libraries

In [3]:
import pandas as pd
import numpy as np
import os
import glob

## Read File

In [4]:
def find_header_row(file_path):
    """
    Doesn't find the header line in:
    """
    column_name = ['DEPARTAMENTO','DEPARTAMENTO '] 
    header_row = None

    df = pd.read_excel(file_path, header=None)

    for i, row in df.iterrows():
        if column_name[0] in row.values or column_name[1] in row.values:
            header_row = i
            break
        if i>20:
            break

    if header_row is None:
        raise ValueError(f"No se encontró la fila de encabezado en el archivo {file_path}")

    return header_row

def read_excel_with_variable_header(file_path):
    header_row = find_header_row(file_path)

    df = pd.read_excel(file_path, header=header_row)

    total_row_index = df[df.iloc[:, 0].str.contains("TOTAL", case=False, na=False)].index.min()

    if total_row_index is not None:
        df = df.iloc[:total_row_index]

    return df

In [5]:
file_paths = glob.glob(os.path.join(PATH_FILE+'/Delitos_Sexuales/','*.xlsx')) + glob.glob(os.path.join(PATH_FILE+'/Delitos_Sexuales/','*.xls'))
dataframes = []

for file_path in file_paths:
    try:
        df = read_excel_with_variable_header(file_path)
        dataframes.append(df)
    except ValueError as e:
        print(e)

## Clean File

In [19]:
dataframes[10]['DEPARTAMENTO_TEMP']=dataframes[10]['ARMAS_MEDIOS']
dataframes[10]['ARMAS_MEDIOS']=dataframes[10]['DEPARTAMENTO']
dataframes[10]['DEPARTAMENTO']=dataframes[10]['DEPARTAMENTO_TEMP']
dataframes[10].drop(columns='DEPARTAMENTO_TEMP',inplace=True)

In [21]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'DELITO', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'DELITO ': 'DELITO',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

standardized_dataframes = [standardize_columns(df) for df in dataframes]

In [22]:
delitos_sexuales_df = pd.concat(standardized_dataframes, ignore_index=True)

In [24]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','CONTUNDENTES','ESCOPOLAMINA','LICOR ADULTERADO','NO REPORTADO',
                             'SIN EMPLEO DE ARMAS','ARMA DE FUEGO','CINTAS/CINTURON','CORTANTES','CORTOPUNZANTES','ESPOSAS','ARMAS BLANCAS']
delitos_sexuales_df["AGRUPA_EDAD_PERSONA"] = delitos_sexuales_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
delitos_sexuales_df["GENERO"] = delitos_sexuales_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
delitos_sexuales_df["ARMAS_MEDIOS"] = delitos_sexuales_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")
delitos_sexuales_df["DELITO"] = delitos_sexuales_df["DELITO"].fillna("NO REPORTADO")

In [27]:
delitos_sexuales_output = delitos_sexuales_df.groupby(["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","DELITO"]).sum().reset_index()

## Export File

In [31]:
delitos_sexuales_output.to_csv(OUTPUT_PATH_FILE+"/delitos_sexuales.csv",index=False)